In [ ]:
import requests
import pandas as pd
import time

# Load the API key
api_key = 'insert_your_api_key_here'

# Verify API Key
if not api_key:
    raise ValueError("API key is not set.")

# List of symbols to retrieve data for
symbols = ['BTG', 'GOLD']

# Function to handle API requests with error handling
def make_api_request(url, symbol, function_name):
    try:
        r = requests.get(url)
    except requests.exceptions.RequestException as e:
        print(f"Request exception for symbol {symbol} in {function_name}: {e}")
        return None

    try:
        data = r.json()
    except ValueError:
        print(f"Error parsing JSON for symbol {symbol} in {function_name}. Response content: {r.text}")
        return None

    if 'Error Message' in data:
        print(f"Error fetching {function_name} for symbol {symbol}: {data['Error Message']}")
        return None
    if 'Note' in data:
        print(f"API Notice for {function_name} with symbol {symbol}: {data['Note']}")
        return None
    if 'Information' in data:
        print(f"API Information for {function_name} with symbol {symbol}: {data['Information']}")
        return None

    return data

# Function to get daily time series data
def get_time_series_daily(symbol):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}'
    data = make_api_request(url, symbol, 'TIME_SERIES_DAILY')
    if data and 'Time Series (Daily)' in data:
        daily_data = data['Time Series (Daily)']
        records = []
        for date, metrics in daily_data.items():
            record = {
                'symbol': symbol,
                'date': date,
                'open': float(metrics['1. open']),
                'high': float(metrics['2. high']),
                'low': float(metrics['3. low']),
                'close': float(metrics['4. close']),
                'volume': int(metrics['5. volume'])
            }
            records.append(record)
        return records
    else:
        print(f"No time series data found for symbol {symbol}")
        return []

# Collect data for each symbol
time_series_data = []

for symbol in symbols:
    data = get_time_series_daily(symbol)
    if data:
        time_series_data.extend(data)
    time.sleep(15)  # Respect API rate limits

# Create a DataFrame from the collected data
time_series_df = pd.DataFrame(time_series_data)
print(time_series_df.head())  # Verify the DataFrame contents

# Save the DataFrame to an Excel file
time_series_df.to_excel('time_series_daily.xlsx', index=False)

print("Done")
